<a href="https://colab.research.google.com/github/asokraju/Rienforcement-Learning/blob/master/David_Silver_Lecture_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Lecture 5: Model Free Control

This Document contains the simulation of examples provided in [Lecture 5](https://youtu.be/0g4j2k_Ggc4?list=PLqYmG7hTraZDM-OYHWgPebj2MfCFzFObQ) of Introduction to Rienforcement Learning by David Silver

Author: Krishna Chaitanya Kosaraju

email: kkrishnachaitanya89@gmail.com

In [0]:
import pandas as pd
import numpy as np
import random
import gym

env = gym.make('Blackjack-v0')

#Part one: MC Q-leaning


In [0]:
# State Action N(state) Q_value 
def eps_greedy(state, Q_funct, n_iter = 1):
  sum_of_cards, _, _= state
  if n_iter <= 10000:
    return random.choice([0,1])
  else:
    if state in Q_funct.keys():
      temp = max(Q_funct[state], key=Q[state].get)
      temp_random = random.choice([0,1])
      return np.random.choice([temp, temp_random], size = 1, p = [1-(1/n_iter), 1/n_iter])[0]
    else:
      return random.choice([0,1])

def episode(Q_funct, n_iter = 1):
  done = False
  state = env.reset()
  Actions = []
  States = []
  Rewards = []
  while not done:
    action = eps_greedy(state, Q_funct, n_iter = n_iter)
    next_state, r, done, _ = env.step(action)
    States.append(state)
    Actions.append(action)
    Rewards.append(r)
    state = next_state
    
  States.append(state)
  Actions.append(eps_greedy(state, Q_funct, n_iter = n_iter))
  return Actions, States, Rewards

def Returns( rewards, gamma =0.5):
  returns = []
  temp =0
  for indx ,r in enumerate(reversed(rewards)):
    temp = r +gamma * temp
    returns.append(temp)
  return list(reversed(returns))



no_episodes = 1000*500
Q = {}
N = {}
for i in range(no_episodes):
  actions,  states, rewards = episode(Q_funct = Q, n_iter = i)
  #print(states)
  #print(actions)
  Gt = Returns(rewards)
  for indx, state in enumerate(states):
    if state not in Q.keys():
      Q[state] = {actions[indx]:0}
      N[state] = {actions[indx]:1}
    else:
      if actions[indx] not in Q[state].keys():
        Q[state][actions[indx]] = 0
        N[state][actions[indx]] = 1

  for indx in range(len(states)-1):
    N[states[indx]][actions[indx]] = N[states[indx]][actions[indx]] + 1
    temp = Gt[indx] + Q[states[indx+1]][actions[indx+1]] - Q[states[indx]][actions[indx]]
    Q[states[indx]][actions[indx]] = Q[states[indx]][actions[indx]] + (1/N[states[indx]][actions[indx]])*temp

print(Q)
print(N)




{(15, 4, False): {0: -0.9859514213315788, 1: 0.20404124483554978}, (14, 10, False): {1: -0.16906720877417897, 0: -1.7027276370628739}, (16, 10, False): {0: -1.87769794603635, 1: -0.22676564819534512}, (6, 10, False): {1: -0.41245989242285347, 0: -1.6901979350000471}, (10, 10, False): {0: -1.2625212603505334, 1: 0.6197872442401877}, (18, 8, False): {1: -0.68629445428731, 0: 0.3302964218510835}, (27, 8, False): {1: 0, 0: 0}, (11, 10, False): {0: -1.7428863964516537, 1: 1.3709175213217362}, (12, 5, False): {1: 0.34708514227311066, 0: -0.9082581890034692}, (22, 5, False): {1: 0, 0: 0}, (13, 4, True): {0: -0.5805898311811566, 1: 0.9668767365573208}, (10, 4, False): {1: 1.295411874790406, 0: -0.6563634031921909}, (19, 4, False): {0: 2.1706500225960377, 1: -0.9228711462120147}, (15, 5, False): {0: -0.8942873273341305, 1: 0.12163166853082534}, (25, 10, False): {0: 0, 1: 0}, (18, 4, False): {1: -0.9103375116218578, 0: 0.8068406406464652}, (23, 4, False): {1: 0, 0: 0}, (13, 10, False): {1: -0.13

#SARSA Learning

In [0]:
def eps_greedy_action(state, Q_funct, n_iter):
  if n_iter <=10000:
    return random.choice([0,1])
  else:
    if state in Q_funct.keys():
      temp = max(Q_funct[state], key = Q_funct[state].get)
      temp_random = random.choice([0,1])
      return np.random.choice([temp, temp_random], size = 1, p = [1-(1/n_iter), 1/n_iter])[0]
    else:
      return random.choice([0,1])

no_episodes = 1000*500
alpha =0.01
Q = {}
N = {}
for i in range(no_episodes):
  done = False
  state = env.reset()
  while not done:
    action = eps_greedy_action(state, Q_funct = Q, n_iter = i)
    next_state, r, done, _ = env.step(action)
    next_action = eps_greedy_action(next_state, Q_funct = Q, n_iter = i)
    if state not in Q.keys():
      Q[state] = {action:0}
    else:
      if action not in Q[state].keys():
        Q[state][action] = 0
    if next_state not in Q.keys():
      Q[next_state] = {next_action:0}
    else:
      if next_action not in Q[next_state].keys():
        Q[next_state][next_action] = 0

    temp = r + Q[next_state][next_action] - Q[state][action]
    Q[state][action] = Q[state][action] + alpha*temp
    state = next_state

In [0]:
Q

# Forward n-step SARSA($\lambda$)

In [0]:
# State Action N(state) Q_value 
def eps_greedy(state, Q_funct, n_iter = 1):
  sum_of_cards, _, _= state
  if n_iter <= 10000:
    return random.choice([0,1])
  else:
    if state in Q_funct.keys():
      temp = max(Q_funct[state], key=Q[state].get)
      temp_random = random.choice([0,1])
      return np.random.choice([temp, temp_random], size = 1, p = [1-(1/n_iter), 1/n_iter])[0]
    else:
      return random.choice([0,1])

def n_step(Q_funct, steps = 1, n_iter = 1):
  done = False
  state = env.reset()
  Actions = []
  States = []
  Rewards = []
  while ((not done) and (n_steps <= 1)):
    action = eps_greedy(state, Q_funct, n_iter = n_iter)
    next_state, r, done, _ = env.step(action)
    States.append(state)
    Actions.append(action)
    Rewards.append(r)
    state = next_state
    
  States.append(state)
  Actions.append(eps_greedy(state, Q_funct, n_iter = n_iter))
  return Actions, States, Rewards

def Returns( rewards, gamma =0.5):
  returns = []
  temp =0
  for indx ,r in enumerate(reversed(rewards)):
    temp = r +gamma * temp
    returns.append(temp)
  return list(reversed(returns))

def lam_Returns(rewards, gamma =0.5, lam = 0.5):
  lam_returns = []
  temp =0
  returns = Returns( rewards, gamma =0.5)
  for indx ,r in enumerate(reversed(returns)):
    temp = r + lam * temp
    lam_returns.append(temp)
  return list(reversed((1-lam)*np.array(lam_returns)))

no_episodes = 1000*500
n_steps = 1
Q = {}
N = {}
for i in range(no_episodes):
  actions,  states, rewards = n_step( Q_funct = Q, steps = n_steps, n_iter = i)
  #print(states)
  #print(actions)
  Gt = lam_Returns(rewards)
  for indx, state in enumerate(states):
    if state not in Q.keys():
      Q[state] = {actions[indx]:0}
    else:
      if actions[indx] not in Q[state].keys():
        Q[state][actions[indx]] = 0

  temp = Gt[0] + Q[states[-1]][actions[-1]] - Q[states[0]][actions[0]]
  Q[states[0]][actions[0]] = Q[states[0]][actions[0]] + alpha*temp

In [0]:
Q

# Backward View of SARSA($\lambda $)

In [0]:
# State Action N(state) Q_value 
def eps_greedy(state, Q_funct, n_iter = 1):
  sum_of_cards, _, _= state
  if n_iter <= 10000:
    return random.choice([0,1])
  else:
    if state in Q_funct.keys():
      temp = max(Q_funct[state], key=Q[state].get)
      temp_random = random.choice([0,1])
      return np.random.choice([temp, temp_random], size = 1, p = [1-(1/n_iter), 1/n_iter])[0]
    else:
      return random.choice([0,1])

no_episodes = 1000*500
alpha =0.01
lam = 0.5
gamma = 1
Q = {}
N = {}
ET = {}
for i in range(no_episodes):
  done = False
  state = env.reset()
  while not done:
    action = eps_greedy_action(state, Q_funct = Q, n_iter = i)
    next_state, r, done, _ = env.step(action)
    next_action = eps_greedy_action(next_state, Q_funct = Q, n_iter = i)
    #if the state and action were first time explored, then
    #initalizing Q and Eligibility trace state-action value function
    # first we initalize Q
    if state not in Q.keys():
      Q[state] = {action:0}
    else:
      if action not in Q[state].keys():
        Q[state][action] = 0
    if next_state not in Q.keys():
      Q[next_state] = {next_action:0}
    else:
      if next_action not in Q[next_state].keys():
        Q[next_state][next_action] = 0
#   Initializing ET
    if state not in ET.keys():
      ET[state] = {action : 0}
    else:
      if action not in ET[state].keys():
        ET[state][action] = 0
    #updating eligibility traces for all the states
    ET[state][action] = ET[state][action] +1
    #TD error
    delta = r + gamma*Q[next_state][next_action] - Q[state][action]
    Q[state][action] = Q[state][action] + alpha*delta*ET[state][action]
    for every_state in ET.keys():
      for every_action in ET[every_state].keys():
        ET[every_state][every_action] = lam*gamma*ET[every_state][every_action]
    state = next_state

print(Q)

{(6, 2, False): {1: 7.880473050915658, 0: -0.060015836946733965}, (16, 2, False): {1: 7.733162658961461, 0: -0.4698777554935089}, (26, 2, False): {1: 0, 0: 0}, (11, 8, False): {0: -0.12556082869283003, 1: 17.860921687191148}, (18, 7, False): {0: 21.155871247590188, 1: -0.2989196294023162}, (6, 5, False): {1: 7.6841243764873335, 0: -0.03696318823621186}, (9, 5, False): {1: 11.49546628383465, 0: -0.009329027781244286}, (13, 1, False): {0: -0.4900682339102095, 1: 3.8830387909579893}, (16, 7, False): {1: 9.920922721528276, 0: -0.3747699737370229}, (26, 7, False): {1: 0, 0: 0}, (20, 10, False): {1: -0.8017045412897428, 0: 131.0413995893625}, (22, 10, False): {1: 0, 0: 0}, (19, 9, False): {1: -0.20309625927073416, 0: 15.202795950632666}, (25, 9, False): {1: 0, 0: 0}, (14, 4, False): {1: 8.347719863992047, 0: -0.32065620386521726}, (16, 4, False): {0: -0.41334094728304843, 1: 5.9616868992216}, (26, 4, False): {1: 0, 0: 0}, (21, 10, True): {0: 82.73034338065295, 1: -0.1428073787559256}, (13, 8